In [ ]:
import pandas as pd
from   bs4 import BeautifulSoup
from random import choice
import requests
from time import sleep
import datetime
import time

In [ ]:



# Helper function to generate dates
def daterange(start_date, end_date):
    for n in range(int((end_date - start_date).days)):
        yield start_date + timedelta(n)

from datetime import date, timedelta
# Start and end dates
start_date = date(2023, 2, 17)
end_date = date(2023, 6, 30)  # today's date

base_url = 'https://www.espn.com/college-baseball/scoreboard/_/date/'
game_data = []  # this list will hold our game data

# Loop over each date
for single_date in tqdm(daterange(start_date, end_date)):
    url = base_url + single_date.strftime("%Y%m%d")
    
    # Try accessing the page
    try:
        response = requests.get(url)
        response.raise_for_status()  # raise exception if invalid response
    except (requests.HTTPError, requests.ConnectionError):
        # Handle the exception if it occurs
        print(f"No data for {single_date.strftime('%Y-%m-%d')}")
        continue

    soup = BeautifulSoup(response.text, 'html.parser')







    # Find all game sections
    game_sections = soup.find_all('section', class_='Scoreboard bg-clr-white flex flex-auto justify-between')

    # For each game section, get the required data
    for section in game_sections:   
        game = {}

        ## Add the date to the game data
        game['date'] = single_date.strftime('%Y-%m-%d')
        
        # Find the link to the box score and extract the game Id
        box_score_link = section.find('a', text='Box Score')
        if box_score_link is not None:
            game['game_id'] = box_score_link['href'].split('/')[-1]

        game_info = section.find('div', class_='ScoreboardScoreCell__Note clr-gray-04 n9 w-auto ml0')
        if game_info is not None:
            game['game_info'] = game_info.text
            
        game_run_elements = section.find_all('div', class_='ScoreboardScoreCell__Value flex justify-center pl2 baseball')
        if game_run_elements is not None and len(game_run_elements) >= 4:  # 4 because we have runs, hits, and errors for both teams
            game['away_team_runs'] = game_run_elements[0].text
            game['home_team_runs'] = game_run_elements[3].text

                    # Add game data to the list
        game_data.append(game)

        sleep(.1)  # sleep for 100 milliseconds to avoid spamming the server





# Convert the list of games to a DataFrame
df = pd.DataFrame(game_data)

# Dropr rows wuth nulls in game id

df = df.dropna(subset=['game_id'])



# Print DataFrame
# print(df)


In [ ]:
## save data for reference
df.to_csv('espn_college_baseball_etl_1.csv', index=False)


In [1]:
## RELOAD THE DATA FOR CLEANING
import pandas as pd
from   bs4 import BeautifulSoup
from random import choice
import requests
from time import sleep
import datetime
import time
import random
from fake_useragent import UserAgent

from tqdm import tqdm

## RELOAD THE DATA FOR CLEANING
df = pd.read_csv('espn_college_baseball_etl_1.csv')

## LOAD DIFFERENT ONE
# df = pd.read_csv('TEMP/')



In [2]:
df.info()

# df.head()

## date histogram
# df['date'].hist(bins=20)

# ## line graph of games per day
# df['date'].value_counts().plot.line()


## create a list from the games_ids
game_ids = df['game_id'].tolist()
# randomize the list
random.shuffle(game_ids)
# Make sure they are strings
game_ids = [str(x) for x in game_ids]



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1190 entries, 0 to 1189
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   date            1190 non-null   object
 1   game_id         1190 non-null   int64 
 2   away_team_runs  1190 non-null   int64 
 3   home_team_runs  1190 non-null   int64 
 4   game_info       229 non-null    object
dtypes: int64(3), object(2)
memory usage: 46.6+ KB


In [3]:
base_game_url = 'https://www.espn.com/college-baseball/boxscore/_/gameId/'
scraper_base = "http://api.scraperapi.com?api_key=7efbd1c22cb922745d8db50ce534d460&url="


######### SETTINGS #########
####################################################################
# List of urls
urls = [scraper_base + base_game_url + game_id for game_id in game_ids]



save_interval = 5
request_counter = 0

# Initialize DataFrame for results
results = pd.DataFrame(columns=['location', 'date', 'date2', 'time', 'team_1', 'team_2', 'runs_1', 'hits_1', 'errors_1', 'home_runs_1', 'runs_2', 'hits_2', 'errors_2', 'home_runs_2'])

# Initialize UserAgent
ua = UserAgent()

# You have to define proxy and headers before using them
proxy = {}  # replace this with your proxy dictionary
headers = {"User-Agent": ua.random}

for url in tqdm(urls):
    try:
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            # Increment the request counter
            request_counter += 1

            soup = BeautifulSoup(response.content, 'html.parser')

            # Load tables from url content
            tables = pd.read_html(response.content)

            # The second table contains the data we're interested in
            score_table = tables[1]

            # The team names are in the first table
            team_table = tables[0]
            team_1 = team_table.iloc[0, 0]
            team_2 = team_table.iloc[1, 0]

            # Extract the runs, hits, and errors for each team
            runs_1, hits_1, errors_1 = score_table.iloc[0, -3:]
            runs_2, hits_2, errors_2 = score_table.iloc[1, -3:]

            # The team's home runs are in tables 3 and 9
            # Convert the 'HR' column to numeric before summing
            home_runs_1 = pd.to_numeric(tables[3]['HR'].iloc[-1])  # Sum the HR column for team 1
            home_runs_2 = pd.to_numeric(tables[5]['HR'].iloc[-1])  # Sum the HR column for team 2
            home_runs = home_runs_1 + home_runs_2

            location = soup.find('div', {'class': 'n6 clr-gray-03 GameInfo__Location__Name--noImg'}).text
            meta = soup.find('div', {'class': 'n8 GameInfo__Meta'}).text.split(',')
            time = meta[0].strip()
            date2 = meta[1].strip()

            # Add the results to the results dataframe
            results = results.append({
                'date2': date2, 'location': location, 'time': time,
                'team_1': team_1, 'team_2': team_2,
                'runs_1': runs_1, 'hits_1': hits_1, 'errors_1': errors_1, 
                'runs_2': runs_2, 'hits_2': hits_2, 'errors_2': errors_2,
                
                'home_runs_1': home_runs_1, 'home_runs_2': home_runs_2, 'home_runs': home_runs,
            }, ignore_index=True)

        else:
            print(f"Error: {response.status_code} - {url}")
            continue

        sleep(.1)  # waits for 1 second

        # If we've hit the saving interval, write the DataFrame to a CSV
        if request_counter % save_interval == 0:
            results.to_csv(f'TEMP/NCAA_Partial/{request_counter}_data_saved.csv', index=False)
            print(f"Data saved to CSV after {request_counter} requests.")

    except Exception as e:
        print(f"Error occurred: {e}")
        continue


    



  0%|          | 0/1190 [00:00<?, ?it/s]C:\Users\Justin\AppData\Local\Temp\ipykernel_20012\1700731311.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
  0%|          | 1/1190 [00:02<49:58,  2.52s/it]C:\Users\Justin\AppData\Local\Temp\ipykernel_20012\1700731311.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
  0%|          | 2/1190 [00:05<59:36,  3.01s/it]C:\Users\Justin\AppData\Local\Temp\ipykernel_20012\1700731311.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
  0%|          | 3/1190 [00:07<46:01,  2.33s/it]C:\Users\Justin\AppData\Local\Temp\ipykernel_20012\1700731311.py:62: FutureWarning: The frame.append method is deprecated a

Data saved to CSV after 5 requests.


C:\Users\Justin\AppData\Local\Temp\ipykernel_20012\1700731311.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
  1%|          | 6/1190 [00:15<59:01,  2.99s/it]C:\Users\Justin\AppData\Local\Temp\ipykernel_20012\1700731311.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
  1%|          | 7/1190 [00:17<51:35,  2.62s/it]C:\Users\Justin\AppData\Local\Temp\ipykernel_20012\1700731311.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
  1%|          | 8/1190 [00:18<42:08,  2.14s/it]C:\Users\Justin\AppData\Local\Temp\ipykernel_20012\1700731311.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a futu

Data saved to CSV after 10 requests.


C:\Users\Justin\AppData\Local\Temp\ipykernel_20012\1700731311.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
  1%|          | 11/1190 [00:26<49:49,  2.54s/it]C:\Users\Justin\AppData\Local\Temp\ipykernel_20012\1700731311.py:62: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({
  1%|          | 12/1190 [00:48<2:46:37,  8.49s/it]

In [ ]:
results.tail()

results.info()

## Sum of home runs
results['home_runs'].sum()

## location value counts
results['location'].value_counts()

In [ ]:
### Save to CSV

## Partial file because of the http error
results.to_csv('TEMP/SUNDAY_NIGHT_collegebaseball_scrape.csv', index=False)